In [15]:
# ==========================================
# PARAMETER CELL (Fabric Notebook)
# ==========================================
# Ces variables seront surchargées par le pipeline.
# valeurs par défaut pour les tests locaux.
#
# run_id:        string (GUID generated by pipeline master)
# entity_code:   string (transactions|fx_rates|mcc|users|cards)
# load_mode:     string (full|incremental) - optional (may be empty)
# as_of_date:    string (YYYY-MM-DD) - optional (may be empty)
# environment:   string (dev|prod)
# pipeline_name: string (pl_silver_load_master)
#
# Example local defaults (ONLY for ad-hoc interactive runs; remove in prod):
from datetime import datetime, timezone
try:
    run_id
except NameError:
    run_id = f"manual-{datetime.now(timezone.utc).strftime('%Y%m%dT%H%M%SZ')}"
    entity_code = "fx_rates"
    load_mode = "full"          # will fallback to ctl default
    as_of_date = ""         # optional
    environment = "dev"
    pipeline_name = "manual"



StatementMeta(, 88a75acd-3d24-4d49-a3fc-2fe2deb1fe0b, 17, Finished, Available, Finished)

In [16]:
# ============================================================
# nb_load_silver  (Microsoft Fabric / Spark Notebook)
# Router + logging wrapper for Silver entity notebooks
#
# IMPORTANT (Fabric parameters):
# - In Fabric, define parameters in the first cell using:
#     run_id, entity_code, load_mode, as_of_date, environment, pipeline_name
# - Then reference them as regular Python variables in subsequent cells.
# ============================================================

import json
import time
from datetime import datetime, timezone
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField,
    StringType, TimestampType,
    LongType, IntegerType
)
import notebookutils
import traceback

# Notebook exit exception can come from different namespaces in Fabric.
# We keep a best-effort import + a name-based fallback.
try:
    from notebookutils.mssparkutils.handlers.notebookHandler import NotebookExit  # type: ignore
except Exception:
    NotebookExit = None  # fallback to name-based detection

def _is_notebook_exit(exc: BaseException) -> bool:
    return exc.__class__.__name__ == "NotebookExit"

# Runtime contract location (Lakehouse Files)
RUNTIME_CONTRACT_PATH = "Files/governance/runtime/silver/entity_payload.json"

# ------------------------------------------------------------
# Tables de contrôle & logging (alignées sur nb_silver_ddl.py)
# ------------------------------------------------------------
CTL_TABLE  = "silver_ctl_entity"
STEP_TABLE = "silver_log_steps"

# Notes:
# - STEP_TABLE contient payload_json (contrat runtime v1.0)
# - CTL_TABLE pilote notebook_path, timeout, critical, etc.

# ----------------------------
# Helpers
# ----------------------------
def utc_now():
    return datetime.now(timezone.utc)

def to_int(x, default=None):
    try:
        return int(x)
    except Exception:
        return default

def safe_str(x, max_len=4000):
    s = "" if x is None else str(x)
    return s[:max_len]

def _read_text(path: str) -> str:
    
    #   Read a small text/JSON file from Lakehouse Files via Fabric utilities.
    #   mssparkutils.fs.open() is not available in some Fabric runtimes.
    #   Use fs.head() instead (sufficient for small contracts).
    
    try:
    # Prefer notebookutils.fs.head (newer namespace), fallback to mssparkutils.fs.head
        try:
            return notebookutils.fs.head(path, 1024 * 1024)  # up to 1MB
        except Exception:
            return mssparkutils.fs.head(path, 1024 * 1024)
    except Exception as e:
        raise FileNotFoundError(f"Cannot read runtime contract at '{path}': {e}")

def load_runtime_contract(path: str = RUNTIME_CONTRACT_PATH) -> dict:
    """
    Load runtime payload contract JSON from Lakehouse Files.
    """
    raw = _read_text(path)
    try:
        return json.loads(raw)
    except Exception as e:
        raise ValueError(f"Runtime contract file is not valid JSON: {path}. Error: {e}")

def _extract_dot(obj: dict, dot_path: str):
    """
    Extract nested value using dot notation (e.g., 'metrics.row_out').
    Returns None if any segment is missing.
    """
    cur = obj
    for seg in dot_path.split("."):
        if cur is None:
            return None
        if isinstance(cur, dict) and seg in cur:
            cur = cur.get(seg)
        else:
            return None
    return cur

def validate_payload_against_contract(payload: dict, contract: dict):
    """
    Enforce required fields defined in the runtime contract.
    The contract is expected to have 'required_fields' as a list of dot-paths.
    """
    required = contract.get("required_fields", [])
    if not isinstance(required, list) or len(required) == 0:
        raise ValueError("Runtime contract is missing a non-empty 'required_fields' list")

    missing = []
    for p in required:
        v = _extract_dot(payload, p) if isinstance(p, str) else None
        if v is None:
            missing.append(p)

    if missing:
        raise ValueError(f"Entity payload missing required fields: {missing}")

def normalize_entity_payload(child_payload: dict) -> dict:
    """
    Backward compatibility:
    - If child notebook returns legacy metrics at root (row_out, etc.), wrap them.
    - If it already follows the v1.0 contract, return as-is.
    """
    if isinstance(child_payload, dict) and "metrics" in child_payload and "table" in child_payload:
        return child_payload

    # Legacy -> contract-like envelope
    now = utc_now()
    return {
        "contract_version": "0.x-legacy",
        "layer": "silver",
        "run_id": child_payload.get("run_id", run_id),
        "entity_code": child_payload.get("entity_code", entity_code),
        "load_mode": child_payload.get("load_mode", None),
        "as_of_date": child_payload.get("as_of_date", None),
        "status": child_payload.get("status", "SUCCESS"),
        "metrics": {
            "row_in": child_payload.get("row_in"),
            "row_out": child_payload.get("row_out"),
            "partition_count": child_payload.get("partition_count", 0),
            "dedup_dropped": child_payload.get("dedup_dropped", 0),
        },
        "table": {
            "target_table": child_payload.get("target_table", None),
            "partition_cols": child_payload.get("partition_cols", []),
        },
        "timing": {
            "started_utc": now.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "ended_utc": now.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "duration_ms": 0,
        }
    }


LOG_STEP_SCHEMA = StructType([
    StructField("run_id",          StringType(),   False),
    StructField("entity_code",     StringType(),   False),
    StructField("notebook_name",   StringType(),   True),
    StructField("start_ts",        TimestampType(),True),
    StructField("end_ts",          TimestampType(),True),
    StructField("status",          StringType(),   False),
    StructField("row_in",          LongType(),     True),
    StructField("row_out",         LongType(),     True),
    StructField("partition_count", IntegerType(),  True),
    StructField("dedup_dropped",   LongType(),     True),
    StructField("error_message",   StringType(),   True),
    StructField("payload_json",    StringType(),   True),
])


def append_log_step(payload: dict):
    # Enforce schema explicitly (avoid Spark inference issues when values are None)
    row = (
        str(payload.get("run_id")),
        str(payload.get("entity_code")),
        payload.get("notebook_name"),
        payload.get("start_ts"),
        payload.get("end_ts"),
        str(payload.get("status")),
        payload.get("row_in"),
        payload.get("row_out"),
        payload.get("partition_count"),
        payload.get("dedup_dropped"),
        payload.get("error_message"),
        payload.get("payload_json"),
    )
    spark.createDataFrame([row], schema=LOG_STEP_SCHEMA) \
         .write.format("delta") \
         .mode("append") \
         .saveAsTable(STEP_TABLE)


def exit_notebook(obj: dict):
    # Fabric supports mssparkutils.notebook.exit
    mssparkutils.notebook.exit(json.dumps(obj))

def debug_step(msg):
    print(f"[DEBUG nb_load_silver] {msg}")

# ----------------------------
# Start
# ----------------------------
step_start_ts = utc_now()
t0 = time.time()

# ------------------------------------------------------------
# Resolve entity_code selection (single entity vs ALL/*)
# ------------------------------------------------------------
entity_code_norm = (entity_code or "").strip()
entity_code_up = entity_code_norm.upper()

is_all = entity_code_up in ("ALL", "*", "")

ctl_base = spark.table(CTL_TABLE)

if is_all:
    # Run all enabled entities ordered by execution_order (if exists)
    # (If execution_order column does not exist, fallback to entity_code ordering)
    cols = [c.lower() for c in ctl_base.columns]
    ctl_enabled = ctl_base.where(F.col("enabled") == F.lit(True))

    if "execution_order" in cols:
        ctl_selected = ctl_enabled.orderBy(F.col("execution_order").asc())
    else:
        ctl_selected = ctl_enabled.orderBy(F.col("entity_code").asc())

    ctl_rows = ctl_selected.collect()
    if not ctl_rows:
        raise ValueError(f"No enabled entities found in {CTL_TABLE}.")
else:
    ctl_selected = (
        ctl_base
        .where(F.upper(F.col("entity_code")) == F.lit(entity_code_up))
        .limit(1)
    )
    ctl_rows = ctl_selected.collect()
    if not ctl_rows:
        raise ValueError(f"Entity not found in {CTL_TABLE}: {entity_code}")

# Convert to list of configs (one per entity)
cfg_list = [r.asDict() for r in ctl_rows]

results = []
any_failed = False
any_failed_critical = False

for cfg in cfg_list:
    # Each iteration runs exactly the same logic as before,
    # but for one entity at a time.
    entity_code_current = cfg.get("entity_code")

    # Reset per-entity timers
    step_start_ts = utc_now()
    t0 = time.time()

    # Skip if disabled (should not happen in ALL since we filtered enabled=true, but keep it safe)
    if not bool(cfg.get("enabled", True)):
        append_log_step({
            "run_id": run_id,
            "entity_code": entity_code_current,
            "notebook_name": cfg.get("notebook_name"),
            "start_ts": step_start_ts,
            "end_ts": utc_now(),
            "status": "SKIPPED",
            "payload_json": json.dumps({"status": "SKIPPED", "reason": "disabled"})
        })
        results.append({"entity_code": entity_code_current, "status": "SKIPPED"})
        continue

    # Resolve load_mode
    resolved_load_mode = (load_mode or "").strip() or (cfg.get("load_mode_default") or "full")

    # Log RUNNING
    append_log_step({
        "run_id": run_id,
        "entity_code": entity_code_current,
        "notebook_name": cfg.get("notebook_name"),
        "start_ts": step_start_ts,
        "status": "RUNNING",
        "payload_json": json.dumps({
            "status": "RUNNING",
            "load_mode": resolved_load_mode,
            "as_of_date": as_of_date,
            "environment": environment,
            "pipeline_name": pipeline_name
        })
    })

    # Execute entity notebook
    try:
        nb_path = cfg.get("notebook_path")
        if not nb_path:
            raise ValueError(f"Missing notebook_path for entity {entity_code_current} in {CTL_TABLE}")

        timeout_min = to_int(cfg.get("timeout_minutes"), default=60)
        timeout_sec = timeout_min * 60
        
        # Parameters passed down to entity notebook
        child_params = {
            "run_id": run_id,
            "entity_code": entity_code_current,
            "load_mode": resolved_load_mode,
            "as_of_date": as_of_date,
            "environment": environment
        }

        # Expect child notebook to exit with a JSON string (entity payload)
        child_res = mssparkutils.notebook.run(nb_path, timeout_sec, child_params)
        child_obj = json.loads(child_res) if child_res else {}

        # Load & apply runtime contract (Files/governance/runtime/silver/entity_payload.json)
        contract = load_runtime_contract(RUNTIME_CONTRACT_PATH)

        # Normalize (supports legacy payloads during transition)
        payload = normalize_entity_payload(child_obj)

        # Validate required fields per contract
        validate_payload_against_contract(payload, contract)

        # Extract metrics consistently from contract payload
        metrics = payload.get("metrics", {}) if isinstance(payload, dict) else {}
        child_status = payload.get("status", "SUCCESS")
        row_in = metrics.get("row_in")
        row_out = metrics.get("row_out")
        partition_count = metrics.get("partition_count")
        dedup_dropped = metrics.get("dedup_dropped")

        # Duration (router-level; entity may also provide its own timing)
        end_ts = utc_now()

        append_log_step({
            "run_id": run_id,
            "entity_code": entity_code_current,
            "notebook_name": cfg.get("notebook_name"),
            "start_ts": step_start_ts,
            "end_ts": end_ts,
            "status": child_status,
            "row_in": row_in,
            "row_out": row_out,
            "partition_count": partition_count,
            "dedup_dropped": dedup_dropped,
            "payload_json": json.dumps(payload)
        })

        # Propagate a non-success status as pipeline-visible exit (and fail fast if critical)
        if child_status not in ("SUCCESS", "SKIPPED"):
            raise ValueError(f"Entity notebook returned non-success status: {child_status}")

        results.append({"entity_code": entity_code_current, "status": child_status, "payload": payload})

    except Exception as e:
        # notebook.exit() lève une exception "NotebookExit" pour arrêter le notebook : c'est NORMAL.
        # Selon le namespace/runtime, la classe exacte peut varier; on détecte aussi par nom.
        if (NotebookExit is not None and isinstance(e, NotebookExit)) or _is_notebook_exit(e):
            raise

        msg = safe_str(e)

        append_log_step({
            "run_id": run_id,
            "entity_code": entity_code_current,
            "notebook_name": cfg.get("notebook_name"),
            "start_ts": step_start_ts,
            "end_ts": utc_now(),
            "status": "FAILED",
            "error_message": msg,
            "payload_json": None
        })

        any_failed = True
        is_critical = bool(cfg.get("critical", True))
        any_failed_critical = any_failed_critical or is_critical

        if is_critical:
            # Fail fast on critical entity
            raise
        else:
            results.append({"entity_code": entity_code_current, "status": "FAILED_NONCRITICAL", "error": msg})
            continue

# Final exit for ALL (or single)
if any_failed_critical:
    raise ValueError("One or more CRITICAL entities failed in ALL run.")
else:
    exit_notebook({
        "status": "SUCCESS" if not any_failed else "SUCCESS_WITH_NONCRITICAL_FAILURES",
        "entity": "ALL" if is_all else entity_code,
        "results": results
    })



StatementMeta(, 88a75acd-3d24-4d49-a3fc-2fe2deb1fe0b, 18, Finished, Available, Finished)

[DEBUG nb_load_silver] Before load_runtime_contract
[DEBUG nb_load_silver] Before normalize_entity_payload
[DEBUG nb_load_silver] Before validate_payload_against_contract
[DEBUG nb_load_silver] Before final SUCCESS log
ExitValue: {"status": "SUCCESS", "entity": "fx_rates", "payload": {"contract_version": "1.0", "layer": "silver", "run_id": "manual-20251219T132920Z", "entity_code": "fx_rates", "load_mode": "full", "as_of_date": null, "status": "SUCCESS", "metrics": {"row_in": 451512, "row_out": 451512, "partition_count": 256, "dedup_dropped": 0}, "table": {"target_table": "silver_fx_rates", "partition_cols": ["fx_month"]}, "timing": {"started_utc": "2025-12-19T14:29:08Z", "ended_utc": "2025-12-19T14:29:36Z", "duration_ms": 27477}, "quality": {"fail_fast_checks": [{"name": "base_currency_is_EUR", "passed": true}, {"name": "natural_keys_not_null", "passed": true}]}, "notes": {"message": null}}}